In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner as kt

# Define function to build the model
def build_model(hp):
    inputs = keras.Input(shape=(256, 256, 3))
    
    # Tune the number of filters in the first Conv2D layer
    filters = hp.Choice('filters', values=[32, 64, 128])
    
    x = layers.Conv2D(filters, (3,3), activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(filters*2, (3,3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D()(x)

    # Tune dropout rate
    dropout_rate = hp.Float('dropout', 0.1, 0.5, step=0.1)
    x = layers.Dropout(dropout_rate)(x)
    
    x = layers.Conv2D(filters*4, (3,3), activation='relu', padding='same')(x)
    x = layers.Conv2D(filters*4, (3,3), activation='relu', padding='same')(x)
    
    x = layers.Conv2DTranspose(filters*2, (3,3), strides=2, activation='relu', padding='same')(x)
    x = layers.Conv2DTranspose(filters, (3,3), strides=2, activation='relu', padding='same')(x)
    
    outputs = layers.Conv2D(1, (1,1), activation='sigmoid', padding='same')(x)
    
    model = keras.Model(inputs, outputs)

    # Tune optimizer choice
    optimizer = hp.Choice('optimizer', values=['adam', 'rmsprop', 'sgd'])

    model.compile(
        optimizer=optimizer,
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )
    
    return model


import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
 
def load_images(image_dir, mask_dir, img_size=(256, 256)):
    images, masks = [], []

    image_files = sorted(os.listdir(image_dir))  # Ensure proper order
    mask_files = sorted(os.listdir(mask_dir))  

    for img_file, mask_file in zip(image_files, mask_files):
        img_path = os.path.join(image_dir, img_file)
        mask_path = os.path.join(mask_dir, mask_file)

        img = cv2.imread(img_path)
        img = cv2.resize(img, img_size) / 255.0  # Normalize

        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)  # Load as grayscale
        mask = cv2.resize(mask, img_size) / 255.0  # Normalize
        mask = np.expand_dims(mask, axis=-1)  # Add channel dimension

        images.append(img)
        masks.append(mask)

    return np.array(images), np.array(masks)

# Paths to your dataset
image_dir = "C:\\PGDAI\\Project\\Self Development\\Try_2\\training\\frames"
mask_dir = "C:\\PGDAI\\Project\\Self Development\\Try_2\\training\\lane-masks"

# Load dataset
X, y = load_images(image_dir, mask_dir)

# Split into training & validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [3]:
# Define tuner
tuner = kt.RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=10,  # Number of different hyperparameter combinations to try
    executions_per_trial=1,  # Number of times each model configuration is trained
    directory="hyperparam_tuning",
    project_name="lane_detection"
)

import numpy as np

# Load dataset
data = np.load("dataset_split.npz")

# Access variables
X_train = data["X_train"]
X_val = data["X_val"]
y_train = data["y_train"]
y_val = data["y_val"]

# print("Dataset loaded successfully!")

# # Load dataset
# X_train, y_train = ...  # Load training data
# X_val, y_val = ...  # Load validation data

# Perform hyperparameter tuning
tuner.search(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

# Get the best model
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.get_best_models(num_models=1)[0]

# Print best hyperparameters
print("Best Hyperparameters:")
print(f"Filters: {best_hyperparameters.get('filters')}")
print(f"Dropout Rate: {best_hyperparameters.get('dropout')}")
print(f"Optimizer: {best_hyperparameters.get('optimizer')}")

# Save best model
best_model.save("best_lane_detection_model.h5")


Trial 10 Complete [00h 01m 03s]

Best val_accuracy So Far: 0.9571406245231628
Total elapsed time: 05h 28m 23s


c:\PGDAI\Project\Self Development\myenv\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 30 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Best Hyperparameters:
Filters: 64
Dropout Rate: 0.4
Optimizer: adam
